In [5]:
import os
import subprocess
import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count

In [6]:
!wget https://github.com/Washington-University/HCPpipelines/blob/master/global/templates/MNI152_T1_1mm.nii.gz

--2022-08-10 07:26:43--  https://github.com/Washington-University/HCPpipelines/blob/master/global/templates/MNI152_T1_1mm.nii.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘MNI152_T1_1mm.nii.gz.3’

MNI152_T1_1mm.nii.g     [    <=>             ] 138.34K   160KB/s    in 0.9s    

2022-08-10 07:26:44 (160 KB/s) - ‘MNI152_T1_1mm.nii.gz.3’ saved [141665]



In [7]:
def plot_middle(data, slice_no=None):
    if not slice_no:
        slice_no = data.shape[-1] // 2
    plt.figure()
    plt.imshow(data[..., slice_no], cmap="gray")
    plt.show()
    return


def registration(src_path, dst_path, ref_path):
    command = ["flirt", "-in", src_path, "-ref", ref_path, "-out", dst_path,
               "-bins", "256", "-cost", "corratio", "-searchrx", "0", "0",
               "-searchry", "0", "0", "-searchrz", "0", "0", "-dof", "12",
               "-interp", "spline"]
    subprocess.call(command, stdout=open(os.devnull, "r"),
                    stderr=subprocess.STDOUT)
    return


def orient2std(src_path, dst_path):
    command = ["fslreorient2std", src_path, dst_path]
    subprocess.call(command)
    return


def create_dir(path):
    if not os.path.isdir(path):
        os.makedirs(path)
    return


def unwarp_main(arg, **kwarg):
    return main(*arg, **kwarg)


def main(src_path, dst_path, ref_path):
    print("Registration on: ", src_path)
    try:
        orient2std(src_path, dst_path)
        registration(dst_path, dst_path, ref_path)
    except RuntimeError:
        print("\tFalied on: ", src_path)

    return

In [8]:
parent_dir = os.getcwd()
data_dir = os.path.join(parent_dir, "data")
data_src_dir = os.path.join(data_dir, "ADNIData")
data_dst_dir = os.path.join(data_dir, "ADNIReg")
create_dir(data_dst_dir)

ref_path = os.path.join(parent_dir,"MNI152_T1_1mm.nii.gz")

data_src_paths, data_dst_paths = [], []
for subject in os.listdir(data_src_dir):
    data_src_paths.append(os.path.join(data_src_dir, subject))
    data_dst_paths.append(os.path.join(data_dst_dir, subject))

# Test
# main(data_src_paths[0], data_dst_paths[0], ref_path)

# Multi-processing
paras = zip(data_src_paths, data_dst_paths,
            [ref_path] * len(data_src_paths))
pool = Pool(processes=cpu_count())
pool.map(unwarp_main, paras)


Registration on: Registration on: Registration on: Registration on: Registration on:     Registration on: Registration on: Registration on: Registration on: /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_012_S_1009_MR_MPR____N3__Scaled_Br_20080224151000732_S43089_I92316.niiRegistration on: /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_002_S_1261_MR_MPR____N3__Scaled_Br_20070807144436756_S27226_I65561.nii/mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_005_S_0448_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070904174004430_S32787_I71302.nii   /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_018_S_0369_MR_MPR____N3__Scaled_Br_20070828110545014_S30432_I70680.nii

 
 /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_016_S_1121_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20090210095958290_S61187_I135570.nii/mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_002_S_0413_MR_MPR____N3__Scal

matrix multiplication: problem with matrix inverse; suggest to use solve() instead
terminate called after throwing an instance of 'std::runtime_error'
  what():  matrix multiplication: problem with matrix inverse; suggest to use solve() instead
Aborted
matrix multiplication: problem with matrix inverse; suggest to use solve() instead
terminate called after throwing an instance of 'std::runtime_error'
  what():  matrix multiplication: problem with matrix inverse; suggest to use solve() instead
Aborted
matrix multiplication: problem with matrix inverse; suggest to use solve() instead
terminate called after throwing an instance of 'std::runtime_error'
  what():  matrix multiplication: problem with matrix inverse; suggest to use solve() instead
Aborted


Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_002_S_1261_MR_MPR____N3__Scaled_Br_20081224110926692_S60591_I132220.nii
Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_018_S_0450_MR_MPR____N3__Scaled_Br_20080626135034172_S51236_I111192.nii
Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_007_S_1206_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070713113644677_S25703_I59955.nii
Registration on:  Registration on: /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_002_S_0413_MR_MPR____N3__Scaled_Br_20070216232854688_S14782_I40657.nii /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_023_S_0376_MR_MPR__GradWarp__B1_Correction__N3__Scaled_2_Br_20081001154518431_S13786_I118882.nii

Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_005_S_0448_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20080611160931996_S50957_I109196.nii
R

matrix multiplication: problem with matrix inverse; suggest to use solve() instead
terminate called after throwing an instance of 'std::runtime_error'
  what():  matrix multiplication: problem with matrix inverse; suggest to use solve() instead
Aborted
matrix multiplication: problem with matrix inverse; suggest to use solve() instead
terminate called after throwing an instance of 'std::runtime_error'
  what():  matrix multiplication: problem with matrix inverse; suggest to use solve() instead
Aborted
matrix multiplication: problem with matrix inverse; suggest to use solve() instead
terminate called after throwing an instance of 'std::runtime_error'
  what():  matrix multiplication: problem with matrix inverse; suggest to use solve() instead
Aborted


 /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_012_S_1009_MR_MPR____N3__Scaled_Br_20090106140504041_S60789_I132800.nii/mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_016_S_1121_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20071229175133442_S25229_I86327.nii

Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_002_S_1018_MR_MPR____N3__Scaled_Br_20090106140133108_S60846_I132795.nii
Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_002_S_0413_MR_MPR____N3__Scaled_Br_20070804124954957_S32945_I64551.nii
Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_005_S_0572_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20071223123153496_S34606_I86030.nii
Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_018_S_0633_MR_MPR____N3__Scaled_Br_20070904192251723_S35155_I71451.nii
Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stagead

...

matrix multiplication: problem with matrix inverse; suggest to use solve() instead
terminate called after throwing an instance of 'std::runtime_error'
  what():  matrix multiplication: problem with matrix inverse; suggest to use solve() instead
Aborted


Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_136_S_0300_MR_MPR____N3__Scaled_Br_20081013164320616_S50330_I120436.nii
Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_136_S_1227_MR_MPR____N3__Scaled_Br_20080321160112071_S46753_I99265.nii
Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_136_S_0429_MR_MPR____N3__Scaled_Br_20071127184003417_S33730_I83549.nii
Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_136_S_1227_MR_MPR____N3__Scaled_Br_20090514132202278_S64957_I143856.nii
Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_136_S_0426_MR_MPR____N3__Scaled_2_Br_20081008134424912_S15017_I119731.nii


matrix multiplication: problem with matrix inverse; suggest to use solve() instead
terminate called after throwing an instance of 'std::runtime_error'
  what():  matrix multiplication: problem with matrix inverse; suggest to use solve() instead
Aborted


Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_136_S_0429_MR_MPR____N3__Scaled_Br_20081013165355471_S51722_I120446.nii
Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_136_S_0426_MR_MPR____N3__Scaled_Br_20070809233020851_S32971_I66787.nii
Registration on:  /mnt/c/Users/hhich/OneDrive/Desktop/stageadni/data/ADNIData/ADNI_136_S_0579_MR_MPR-R____N3__Scaled_2_Br_20081008134806053_S16545_I119735.nii


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,